# Kepler Framework Examples

In [ ]:
from kepler.pandas.menu       import ElectronSequence as Chain
from kepler.pandas.readers    import load, load_in_loop
from kepler.pandas.decorators import create_ringer_v8_decorators, create_ringer_v9_decorators, RingerDecorator
from kepler.pandas.decorators import create_ringer_v8_new_decorators, create_ringer_v8_half_fast_decorators, create_ringer_v8_34_decorators, create_ringer_v8_half_decorators

import kepler
import tqdm
import rootplotlib as rpl
import mplhep as hep
import root_numpy
import ROOT
ROOT.gStyle.SetOptStat(0);
import array

import numpy as np
import pandas as pd
import collections
import os
from pprint import pprint
from copy import deepcopy
import gc


import matplotlib.pyplot as plt
from matplotlib import gridspec
%matplotlib inline

import mplhep as hep

import warnings
warnings.filterwarnings('ignore')
plt.style.use(hep.style.ROOT)
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

Welcome to JupyROOT 6.16/00
Using all sub packages with ROOT dependence


# Running for Zee samples

In [ ]:
## codigo original
# dpath = '/home/jodafons/public/cern_data/new_files/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.25bins'
# dpath+= '/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.25bins_et{ET}_eta{ETA}.npz'
# paths = []
# for et in range(5):
#     for eta in range(5):
#         paths.append( dpath.format(ET=et,ETA=eta) )

real_run = False

## Load Data

In [ ]:
# codigo modificado
dpath = '/home/pedro.lima/data/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.25bins'
dpath+= '/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.25bins_et{ET}_eta{ETA}.npz'
#                                               \
#                                           very loose
paths = []

dev = False
#    False carrega todas as regioes

if dev:
    range_et = [4]   #leblon = 4
    range_eta = [0]  #leblon = 0
else:
    range_et = range(5)
    range_eta = range(5)

for et in range_et:
    for eta in range_eta:
        paths.append( dpath.format(ET=et,ETA=eta) )

In [ ]:
drop_columns = [
                    'RunNumber', 
                    #'avgmu',
                    #'trig_L2_cl_et',
                    #'trig_L2_cl_eta',
                    #'trig_L2_cl_phi',
                    #'trig_L2_cl_reta',
                    #'trig_L2_cl_ehad1', 
                    #'trig_L2_cl_eratio',
                    #'trig_L2_cl_f1', 
                    #'trig_L2_cl_f3', 
                    #'trig_L2_cl_weta2', 
                    #'trig_L2_cl_wstot', 
                    'trig_L2_cl_e2tsts1',
                    'trig_L2_el_hastrack',
                    #'trig_L2_el_pt',
                    'trig_L2_el_eta',
                    'trig_L2_el_phi',
                    'trig_L2_el_caloEta',
                    'trig_L2_el_trkClusDeta',
                    'trig_L2_el_trkClusDphi',
                    'trig_L2_el_etOverPt',
                    'trig_EF_cl_hascluster',
                    #'trig_EF_cl_et',
                    'trig_EF_cl_eta',
                    'trig_EF_cl_etaBE2',
                    'trig_EF_cl_phi',     
                    'trig_EF_el_hascand',
                    #'trig_EF_el_et',
                    'trig_EF_el_eta',
                    'trig_EF_el_etaBE2',
                    'trig_EF_el_phi',
                    'trig_EF_el_rhad1',
                    'trig_EF_el_rhad',
                    'trig_EF_el_f3',
                    'trig_EF_el_weta2',
                    'trig_EF_el_rphi',
                    'trig_EF_el_reta',
                    'trig_EF_el_wtots1',
                    'trig_EF_el_eratio',
                    'trig_EF_el_f1',
                    'trig_EF_el_hastrack',
                    'trig_EF_el_deltaEta1',
                    'trig_EF_el_deltaPhi2',
                    'trig_EF_el_deltaPhi2Rescaled',
                    #'trig_EF_el_lhtight',
                    #'trig_EF_el_lhmedium',
                    #'trig_EF_el_lhloose',
                    #'trig_EF_el_lhvloose', 
                    # Offline variables
                    #'el_et',
                    #'el_eta',
                    'el_etaBE2',
                    #'el_phi',
                    # offline shower shapers
                    #'el_rhad1',
                    #'el_rhad',
                    #'el_f3',
                    #'el_weta2',
                    #'el_rphi',
                    #'el_reta',
                    #'el_wtots1',
                    #'el_eratio',
                    #'el_f1',
                    # offline track
                    #'el_hastrack',
                    'el_numberOfBLayerHits',
                    'el_numberOfPixelHits',
                    'el_numberOfTRTHits',
                    #'el_d0',
                    #'el_d0significance',
                    #'el_eProbabilityHT',
                    'el_trans_TRT_PID',
                    #'el_deltaEta1',
                    'el_deltaPhi2',
                    #'el_deltaPhi2Rescaled',
                    #'el_deltaPOverP',
                    #'el_lhtight',
                    #'el_lhmedium',
                    #'el_lhloose',
                    #'el_lhvloose',
                    'el_TaP_Mass',
                    #'el_TaP_deltaR',
                ] 

# variaveis dos aneis...para plotar o perfil médio, preciso deixar
#drop_columns.extend( ['trig_L2_cl_ring_%d'%i for i in range(100)] )

In [ ]:
os.environ['RINGER_TUNING_PATH']='/home/pedro.lima/workspace/CERN-ATLAS-Qualify-1/analysis/tunings'

decorators = create_ringer_v8_new_decorators()
decorators += create_ringer_v8_half_fast_decorators()
#decorators+= create_ringer_v9_decorators()
decorators += create_ringer_v8_34_decorators()

2022-08-04 13:50:40,802 | Py.RingerDecorator                      INFO Reading... /home/pedro.lima/workspace/CERN-ATLAS-Qualify-1/analysis/tunings/TrigL2_20180125_v8/ElectronRingerTightTriggerConfig.conf
2022-08-04 13:50:41,386 | Py.RingerDecorator                      INFO Reading... /home/pedro.lima/workspace/CERN-ATLAS-Qualify-1/analysis/tunings/TrigL2_20180125_v8/ElectronRingerMediumTriggerConfig.conf
2022-08-04 13:50:41,800 | Py.RingerDecorator                      INFO Reading... /home/pedro.lima/workspace/CERN-ATLAS-Qualify-1/analysis/tunings/TrigL2_20180125_v8/ElectronRingerLooseTriggerConfig.conf
2022-08-04 13:50:42,194 | Py.RingerDecorator                      INFO Reading... /home/pedro.lima/workspace/CERN-ATLAS-Qualify-1/analysis/tunings/TrigL2_20180125_v8/ElectronRingerVeryLooseTriggerConfig.conf
2022-08-04 13:50:42,601 | Py.RingerDecorator                      INFO Reading... /home/pedro.lima/workspace/CERN-ATLAS-Qualify-1/analysis/tunings/TrigL2_20210907_v8-1/ElectronRin

2022-08-04 13:50:40.824762: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-08-04 13:50:40.824802: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: UNKNOWN ERROR (-1)
2022-08-04 13:50:40.824824: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (caloba52): /proc/driver/nvidia/version does not exist
2022-08-04 13:50:40.825033: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-04 13:50:40.856929: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2400000000 Hz
2022-08-04 13:50:40.865469: I tensorflow/compiler/xla/service/service.c

## Setup Chains

ivarloose - é o isolamento loose. Olhar https://twiki.cern.ch/twiki/bin/view/Atlas/TriggerNamingRun2    https://twiki.cern.ch/twiki/bin/view/Atlas/TriggerMenuConvention

ivarloose (Run2 que deve ser mantido para o Run3)- HLT isolation: ptvarcone20/ET<0.1

O pid_name da cadeia está marcado no `lh*` , onde * é o pid_name (ponto de operação da cadeia)

In [ ]:
# create my chain
chains = [
            Chain( "HLT_e24_lhtight_nod0_noringer_ivarloose" , L1Seed = 'L1_EM22VHI'),
            Chain( "HLT_e24_lhtight_nod0_ringer_v8_new_ivarloose", L1Seed = 'L1_EM22VHI' , l2calo_column = 'ringer_v8_new_tight' ),
            Chain( "HLT_e24_lhtight_nod0_ringer_v8_half_ivarloose", L1Seed = 'L1_EM22VHI' , l2calo_column = 'ringer_v8_half_tight' ),
            Chain( "HLT_e24_lhtight_nod0_ringer_v8_34_ivarloose", L1Seed = 'L1_EM22VHI' , l2calo_column = 'ringer_v8_34_tight' ),

            Chain( "HLT_e26_lhtight_nod0_noringer_ivarloose" , L1Seed = 'L1_EM22VHI'),
            Chain( "HLT_e26_lhtight_nod0_ringer_v8_new_ivarloose", L1Seed = 'L1_EM22VHI' , l2calo_column = 'ringer_v8_new_tight' ),
            Chain( "HLT_e26_lhtight_nod0_ringer_v8_half_ivarloose", L1Seed = 'L1_EM22VHI' , l2calo_column = 'ringer_v8_half_tight' ),
            Chain( "HLT_e26_lhtight_nod0_ringer_v8_34_ivarloose", L1Seed = 'L1_EM22VHI' , l2calo_column = 'ringer_v8_34_tight' ),

            Chain( "HLT_e60_lhmedium_nod0_noringer_L1EM24VHI" , L1Seed = 'L1_EM24VHI'),
            Chain( "HLT_e60_lhmedium_nod0_ringer_v8_new_L1EM24VHI", L1Seed = 'L1_EM24VHI' , l2calo_column = 'ringer_v8_new_medium'),
            Chain( "HLT_e60_lhmedium_nod0_ringer_v8_half_L1EM24VHI", L1Seed = 'L1_EM24VHI' , l2calo_column = 'ringer_v8_half_medium'),
            Chain( "HLT_e60_lhmedium_nod0_ringer_v8_34_L1EM24VHI", L1Seed = 'L1_EM24VHI' , l2calo_column = 'ringer_v8_34_medium'),

          
            Chain( "HLT_e140_lhloose_nod0_noringer"  , L1Seed = 'L1_EM24VHI'),
            Chain( "HLT_e140_lhloose_nod0_ringer_v8_new" , L1Seed = 'L1_EM24VHI', l2calo_column = 'ringer_v8_new_loose'),
            Chain( "HLT_e140_lhloose_nod0_ringer_v8_half" , L1Seed = 'L1_EM24VHI', l2calo_column = 'ringer_v8_half_loose'),
            Chain( "HLT_e140_lhloose_nod0_ringer_v8_34" , L1Seed = 'L1_EM24VHI', l2calo_column = 'ringer_v8_34_loose'),
]

OrderedDict([('L1Seed', 'L1_EM22VHI'),
             ('L2Calo', (21000.0, 'trig_L2_cl_lhtight_et22toInf')),
             ('L2', 'trig_L2_el_cut_pt20to50'),
             ('EFCalo', 24000.0),
             ('HLT', (24000.0, 'trig_EF_el_lhtight_ivarloose'))])
OrderedDict([('L1Seed', 'L1_EM22VHI'),
             ('L2Calo', (21000.0, 'ringer_v8_new_tight')),
             ('L2', 'trig_L2_el_cut_pt20to50'),
             ('EFCalo', 24000.0),
             ('HLT', (24000.0, 'trig_EF_el_lhtight_ivarloose'))])
OrderedDict([('L1Seed', 'L1_EM22VHI'),
             ('L2Calo', (21000.0, 'ringer_v8_half_tight')),
             ('L2', 'trig_L2_el_cut_pt20to50'),
             ('EFCalo', 24000.0),
             ('HLT', (24000.0, 'trig_EF_el_lhtight_ivarloose'))])
OrderedDict([('L1Seed', 'L1_EM22VHI'),
             ('L2Calo', (21000.0, 'ringer_v8_34_tight')),
             ('L2', 'trig_L2_el_cut_pt20to50'),
             ('EFCalo', 24000.0),
             ('HLT', (24000.0, 'trig_EF_el_lhtight_ivarloose'))])
Ordered

## Read all bins

In [ ]:
table = load_in_loop( paths, drop_columns=drop_columns, decorators=decorators, chains=chains )

Reading files...: 100%|██████████| 25/25 [1:01:31<00:00, 147.66s/it]


In [ ]:
if True:
    print(table.columns.to_list())

['avgmu', 'trig_L2_cl_et', 'trig_L2_cl_eta', 'trig_L2_cl_phi', 'trig_L2_cl_reta', 'trig_L2_cl_ehad1', 'trig_L2_cl_eratio', 'trig_L2_cl_f1', 'trig_L2_cl_f3', 'trig_L2_cl_weta2', 'trig_L2_cl_wstot', 'trig_L2_cl_ring_0', 'trig_L2_cl_ring_1', 'trig_L2_cl_ring_2', 'trig_L2_cl_ring_3', 'trig_L2_cl_ring_4', 'trig_L2_cl_ring_5', 'trig_L2_cl_ring_6', 'trig_L2_cl_ring_7', 'trig_L2_cl_ring_8', 'trig_L2_cl_ring_9', 'trig_L2_cl_ring_10', 'trig_L2_cl_ring_11', 'trig_L2_cl_ring_12', 'trig_L2_cl_ring_13', 'trig_L2_cl_ring_14', 'trig_L2_cl_ring_15', 'trig_L2_cl_ring_16', 'trig_L2_cl_ring_17', 'trig_L2_cl_ring_18', 'trig_L2_cl_ring_19', 'trig_L2_cl_ring_20', 'trig_L2_cl_ring_21', 'trig_L2_cl_ring_22', 'trig_L2_cl_ring_23', 'trig_L2_cl_ring_24', 'trig_L2_cl_ring_25', 'trig_L2_cl_ring_26', 'trig_L2_cl_ring_27', 'trig_L2_cl_ring_28', 'trig_L2_cl_ring_29', 'trig_L2_cl_ring_30', 'trig_L2_cl_ring_31', 'trig_L2_cl_ring_32', 'trig_L2_cl_ring_33', 'trig_L2_cl_ring_34', 'trig_L2_cl_ring_35', 'trig_L2_cl_ring_36',

In [ ]:
table.head()

,avgmu,trig_L2_cl_et,trig_L2_cl_eta,trig_L2_cl_phi,trig_L2_cl_reta,trig_L2_cl_ehad1,trig_L2_cl_eratio,trig_L2_cl_f1,trig_L2_cl_f3,trig_L2_cl_weta2,...,L1Calo_e140_lhloose_nod0_ringer_v8_half,L2Calo_e140_lhloose_nod0_ringer_v8_half,L2_e140_lhloose_nod0_ringer_v8_half,EFCalo_e140_lhloose_nod0_ringer_v8_half,HLT_e140_lhloose_nod0_ringer_v8_half,L1Calo_e140_lhloose_nod0_ringer_v8_34,L2Calo_e140_lhloose_nod0_ringer_v8_34,L2_e140_lhloose_nod0_ringer_v8_34,EFCalo_e140_lhloose_nod0_ringer_v8_34,HLT_e140_lhloose_nod0_ringer_v8_34
0,44.700001,19273.964844,0.099392,-2.523894,0.972962,-190.000000,0.948686,0.388993,0.008554,0.009378,...,False,False,False,False,False,False,False,False,False,False
1,44.700001,19330.248047,0.264886,-1.784419,0.953338,-39.500000,0.948622,0.321716,0.003106,0.009217,...,False,False,False,False,False,False,False,False,False,False
2,44.700001,15270.058594,-0.249952,-0.758099,1.013074,-249.500000,0.883187,0.487872,0.002307,0.008863,...,False,False,False,False,False,False,False,False,False,False
3,44.630001,17340.074219,0.582413,-1.097990,1.028176,148.000000,0.926765,0.426812,-0.000522,0.010547,...,False,False,False,False,False,False,False,False,False,False
4,44.580002,15489.009766,-0.697643,-2.718542,1.061034,711.230164,0.974475,0.404273,-0.000863,0.009515,...,False,False,False,False,False,False,False,False,False,False


## Energy profiles

In [ ]:
if False:
    normalizar = True
    
    rings=table.columns.values[11:111]
    n_rings = np.linspace(0, 99, 100)
    ee_value = []
    jet_value = []
    layers = [8-1, 8+64-1, 8+64+8-1, 8+64+8+8-1, 8+64+8+8+4-1, 8+64+8+8+4+4-1]
    
    ees = table.loc[(table.target==1)]
    jets = table.loc[(table.target==0)]
    
    for ring in rings:
        ee_value.append(ees[ring].mean())
        jet_value.append(jets[ring].mean())
    
    if normalizar:
        ee_value = [i/sum(ee_value) for i in ee_value]
        jet_value = [i/sum(jet_value) for i in jet_value]
        
    plt.figure(figsize=(25,10))
    plt.plot(n_rings, ee_value, label = 'Electrons')
    plt.plot(n_rings, jet_value, label = 'Jets')
    plt.vlines(x=layers, ymin=0, ymax=0.5, colors = 'black', ls = '--')
    plt.legend(loc = 9)
    plt.grid()
    plt.show()

In [ ]:
if False:  
    def norm1( data ):
        norms = np.abs( data.sum(axis=1) )
        norms[norms==0] = 1
        return data/norms[:,None]
    
    normalizar = True
    
    rings=table.columns.values[11:111]
    n_rings = np.linspace(0, 99, 100)
    ee_value = []
    layers = [8-1, 8+64-1, 8+64+8-1, 8+64+8+8-1, 8+64+8+8+4-1, 8+64+8+8+4+4-1]
    
    ees = table.loc[(table.target==1)]
    ees = ees[rings]
    #jets = table.loc[(table.target==0)]
    
    if normalizar:
        ees = norm1(ees)
        
    plt.figure(figsize=(25,10))
    for irow in range(20000):
        #               |
        #    numero de linhas a processar
        #    todas = len(ees)
        row = ees.iloc[irow]
        plt.plot(n_rings, row, color = 'black', linewidth=0.2)
    plt.vlines(x=layers, ymin=-1, ymax=1, colors = 'black', ls = '--')
    plt.title('Electrons')
    plt.xlabel('Rings', loc='center')
    plt.ylabel('Normalized Energy', loc='center')
    #plt.legend(loc = 9)
    ax = plt.gca()
    ax.axes.xaxis.set_ticklabels([])
    ax.text(2-0.5, 0.7, 'PS', fontsize = 15)
    ax.text(8-0.5, 0.7, 'EM1', fontsize = 15)
    ax.text(8+64-0.5, 0.7, 'EM2', fontsize = 15)
    ax.text(8+64+8-0.5, 0.7, 'EM3', fontsize = 15)
    ax.text(8+64+8+8-0.5, 0.7, 'HAD1', fontsize = 15)
    ax.text(8+64+8+8+4-0.5, 0.7, 'HAD2', fontsize = 15)
    ax.text(8+64+8+8+4+4-0.5, 0.7, 'HAD3', fontsize = 15)
    plt.ylim(-0.07, 0.75)
    plt.xlim(0,99)
    plt.grid()
    plt.savefig('/home/pedro.lima/workspace/CERN-ATLAS-Qualify-1/analysis/kepler/kepler_imgs/Electrons_energy_profile_et%d_eta%d.png'%(range_et[0], range_eta[0]))
    plt.savefig('/home/pedro.lima/workspace/CERN-ATLAS-Qualify-1/analysis/kepler/kepler_imgs/Electrons_energy_profile_et%d_eta%d.pdf'%(range_et[0], range_eta[0]))
    plt.show()

In [ ]:
if False:  
    def norm1( data ):
        norms = np.abs( data.sum(axis=1) )
        norms[norms==0] = 1
        return data/norms[:,None]
    
    normalizar = True
    
    rings=table.columns.values[11:111]
    n_rings = np.linspace(0, 99, 100)
    jet_value = []
    layers = [8-1, 8+64-1, 8+64+8-1, 8+64+8+8-1, 8+64+8+8+4-1, 8+64+8+8+4+4-1]
    
    jets = table.loc[(table.target==0)]
    jets = jets[rings]
    
    if normalizar:
        jets = norm1(jets)
        
    plt.figure(figsize=(25,10))
    for irow in range(20000):
        #               |
        #    numero de linhas a processar
        #    todas = len(jets)
        row = jets.iloc[irow]
        plt.plot(n_rings, row, color = 'black', linewidth=0.2)
    plt.vlines(x=layers, ymin=-1, ymax=1, colors = 'black', ls = '--')
    plt.title('Jets')
    plt.xlabel('Rings', loc='center')
    plt.ylabel('Normalized Energy', loc='center')
    #plt.legend(loc = 9)
    ax = plt.gca()
    ax.axes.xaxis.set_ticklabels([])
    ax.text(2-0.5, 0.7, 'PS', fontsize = 15)
    ax.text(8-0.5, 0.7, 'EM1', fontsize = 15)
    ax.text(8+64-0.5, 0.7, 'EM2', fontsize = 15)
    ax.text(8+64+8-0.5, 0.7, 'EM3', fontsize = 15)
    ax.text(8+64+8+8-0.5, 0.7, 'HAD1', fontsize = 15)
    ax.text(8+64+8+8+4-0.5, 0.7, 'HAD2', fontsize = 15)
    ax.text(8+64+8+8+4+4-0.5, 0.7, 'HAD3', fontsize = 15)
    plt.ylim(-0.07, 0.75)
    plt.xlim(0,99)
    plt.grid()
    plt.savefig('/home/pedro.lima/workspace/CERN-ATLAS-Qualify-1/analysis/kepler/kepler_imgs/Jets_energy_profile_et%d_eta%d.png'%(range_et[0], range_eta[0]))
    plt.savefig('/home/pedro.lima/workspace/CERN-ATLAS-Qualify-1/analysis/kepler/kepler_imgs/Jets_energy_profile_et%d_eta%d.pdf'%(range_et[0], range_eta[0]))
    plt.show()

In [ ]:
if False:  
    def norm1( data ):
        norms = np.abs( data.sum(axis=1) )
        norms[norms==0] = 1
        return data/norms[:,None]
    
    normalizar = True
    
    for iet in range(5):
        for ieta in range(5):
            print('Processing ET: %d ETA: %d' % (iet, ieta))
            
            paths = []
            paths.append( dpath.format(ET=iet,ETA=ieta))
            table = []
            table = load_in_loop( paths, drop_columns=drop_columns, decorators=decorators, chains=chains )
            
            rings=table.columns.values[11:111]
            n_rings = np.linspace(0, 99, 100)
            ee_value = []
            layers = [8-1, 8+64-1, 8+64+8-1, 8+64+8+8-1, 8+64+8+8+4-1, 8+64+8+8+4+4-1]

            ees = table.loc[(table.target==1)]
            ees = ees[rings]
            #jets = table.loc[(table.target==0)]

            if normalizar:
                ees = norm1(ees)

            plt.figure(figsize=(25,10))
            for irow in range(20000):
                #               |
                #    numero de linhas a processar
                #    todas = len(ees)
                row = ees.iloc[irow]
                plt.plot(n_rings, row, color = 'black', linewidth=0.2)
            plt.vlines(x=layers, ymin=-1, ymax=1, colors = 'black', ls = '--')
            plt.title('Electrons')
            plt.xlabel('Rings', loc='center')
            plt.ylabel('Normalized Energy', loc='center')
            #plt.legend(loc = 9)
            ax = plt.gca()
            ax.axes.xaxis.set_ticklabels([])
            ax.text(2-0.5, 0.7, 'PS', fontsize = 15)
            ax.text(8-0.5, 0.7, 'EM1', fontsize = 15)
            ax.text(8+64-0.5, 0.7, 'EM2', fontsize = 15)
            ax.text(8+64+8-0.5, 0.7, 'EM3', fontsize = 15)
            ax.text(8+64+8+8-0.5, 0.7, 'HAD1', fontsize = 15)
            ax.text(8+64+8+8+4-0.5, 0.7, 'HAD2', fontsize = 15)
            ax.text(8+64+8+8+4+4-0.5, 0.7, 'HAD3', fontsize = 15)
            plt.ylim(-0.07, 0.75)
            plt.xlim(0,99)
            plt.grid()
            plt.savefig('/home/pedro.lima/workspace/CERN-ATLAS-Qualify-1/analysis/kepler/kepler_imgs/Electrons_energy_profile_et%d_eta%d.png'%(iet, ieta))
            plt.savefig('/home/pedro.lima/workspace/CERN-ATLAS-Qualify-1/analysis/kepler/kepler_imgs/Electrons_energy_profile_et%d_eta%d.pdf'%(iet, ieta))
            #plt.show()

In [ ]:
if False:  
    def norm1( data ):
        norms = np.abs( data.sum(axis=1) )
        norms[norms==0] = 1
        return data/norms[:,None]
    
    normalizar = True
    
    for iet in range(5):
        for ieta in range(5):
            print('Processing ET: %d ETA: %d' % (iet, ieta))
            
            paths = []
            paths.append( dpath.format(ET=iet,ETA=ieta))
            table = []
            table = load_in_loop( paths, drop_columns=drop_columns, decorators=decorators, chains=chains )
            
            rings=table.columns.values[11:111]
            n_rings = np.linspace(0, 99, 100)
            ee_value = []
            layers = [8-1, 8+64-1, 8+64+8-1, 8+64+8+8-1, 8+64+8+8+4-1, 8+64+8+8+4+4-1]

            jets = table.loc[(table.target==0)]
            jets = jets[rings]

            if normalizar:
                jets = norm1(jets)

            plt.figure(figsize=(25,10))
            
            if len(jets) < 20000:
                size = len(jets)
            else:
                size = 20000
                
            for irow in range(size):
                #               |
                #    numero de linhas a processar
                #    todas = len(jets)
                row = jets.iloc[irow]
                plt.plot(n_rings, row, color = 'black', linewidth=0.2)
            plt.vlines(x=layers, ymin=-1, ymax=1, colors = 'black', ls = '--')
            plt.title('Jets')
            plt.xlabel('Rings', loc='center')
            plt.ylabel('Normalized Energy', loc='center')
            #plt.legend(loc = 9)
            ax = plt.gca()
            ax.axes.xaxis.set_ticklabels([])
            ax.text(2-0.5, 0.7, 'PS', fontsize = 15)
            ax.text(8-0.5, 0.7, 'EM1', fontsize = 15)
            ax.text(8+64-0.5, 0.7, 'EM2', fontsize = 15)
            ax.text(8+64+8-0.5, 0.7, 'EM3', fontsize = 15)
            ax.text(8+64+8+8-0.5, 0.7, 'HAD1', fontsize = 15)
            ax.text(8+64+8+8+4-0.5, 0.7, 'HAD2', fontsize = 15)
            ax.text(8+64+8+8+4+4-0.5, 0.7, 'HAD3', fontsize = 15)
            plt.ylim(-0.07, 0.75)
            plt.xlim(0,99)
            plt.grid()
            plt.savefig('/home/pedro.lima/workspace/CERN-ATLAS-Qualify-1/analysis/kepler/kepler_imgs/Jets_energy_profile_et%d_eta%d.png'%(iet, ieta))
            plt.savefig('/home/pedro.lima/workspace/CERN-ATLAS-Qualify-1/analysis/kepler/kepler_imgs/Jets_energy_profile_et%d_eta%d.pdf'%(iet, ieta))
            #plt.show()